# Data Cleaning Functions Summary

## Overview
This notebook provides comprehensive data cleaning functions for combining and standardizing data from two sources:

1. **Google Calendar data** (from `google_calendar.py`)
2. **Web scraping data** (from `eventsWebscrapping_dfp_V2.ipynb`)

## Available Functions

### `clean_google_calendar_df(df)`
- **Input**: Raw Google Calendar DataFrame with columns: `Summary`, `Start`, `End`, `Location`, `Description`
- **Output**: Standardized DataFrame with columns: `title`, `start_datetime`, `end_datetime`, `location`, `description`, `source`
- **Features**: 
  - Handles missing values
  - Normalizes datetime formats to UTC
  - Standardizes column names
  - Adds source identifier

### `clean_web_scraping_df(df)`
- **Input**: Raw web scraping DataFrame with columns: `title`, `link`, `date_time`, `venue`, `address`
- **Output**: Standardized DataFrame with columns: `title`, `start_datetime`, `end_datetime`, `location`, `description`, `source`
- **Features**:
  - Parses complex datetime strings (including range formats like "start → end")
  - Combines venue and address information into location
  - Handles both dictionary and string address formats
  - Uses link as description
  - Adds source identifier

### `combine_and_clean_dataframes(google_df, web_scraping_df)`
- **Input**: Both raw DataFrames (either can be None)
- **Output**: Combined and cleaned DataFrame
- **Features**:
  - Cleans both DataFrames using appropriate functions
  - Combines into single DataFrame
  - Sorts by start datetime
  - Removes duplicates
  - Returns empty DataFrame with proper columns if no data provided

## Usage Examples

```python
# For Google Calendar data
google_events = get_calendar_events(creds)  # Your existing function
cleaned_google = clean_google_calendar_df(google_events)

# For web scraping data  
web_events = pd.DataFrame(events)  # Your existing scraping results
cleaned_web = clean_web_scraping_df(web_events)

# To combine both sources
combined_events = combine_and_clean_dataframes(google_events, web_events)
```

## Output Schema
All cleaned DataFrames have the following standardized columns:
- `title`: Event name/title
- `start_datetime`: Event start time (UTC normalized)
- `end_datetime`: Event end time (UTC normalized, may be null)
- `location`: Venue/location information
- `description`: Event description or additional details
- `source`: Data source identifier ('google_calendar' or 'web_scraping')

## Data Quality Features
- Removes events with invalid start times
- Handles missing values appropriately
- Normalizes datetime formats and timezones
- Combines location information intelligently
- Maintains data lineage with source tracking

Data Cleaning & Standardizing Functions:

1. clean_google_calendar_df(df) - Clean Google Calendar data
2. clean_web_scraping_df(df) - Clean web scraping data
3. combine_and_clean_dataframes(google_df, web_scraping_df) - Clean and combine both


In [18]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import json

def clean_google_calendar_df(df):
    """
    Clean and standardize Google Calendar DataFrame
    
    Parameters:
    df (pd.DataFrame): Raw Google Calendar DataFrame with columns: Summary, Start, End, Location, Description
    
    Returns:
    pd.DataFrame: Cleaned DataFrame with standardized columns
    """
    if df.empty:
        return pd.DataFrame(columns=['title', 'start_datetime', 'end_datetime', 'location', 'description', 'source'])
    
    # Create a copy to avoid modifying original
    cleaned_df = df.copy()
    
    # Standardize column names
    cleaned_df = cleaned_df.rename(columns={
        'Summary': 'title',
        'Start': 'start_datetime',
        'End': 'end_datetime',
        'Location': 'location',
        'Description': 'description'
    })
    
    # Clean title
    cleaned_df['title'] = cleaned_df['title'].fillna('Untitled Event')
    cleaned_df['title'] = cleaned_df['title'].str.strip()
    
    # Parse and standardize datetime columns with UTC handling
    for col in ['start_datetime', 'end_datetime']:
        if col in cleaned_df.columns:
            cleaned_df[col] = pd.to_datetime(cleaned_df[col], errors='coerce', utc=True)
    
    # Clean location
    cleaned_df['location'] = cleaned_df['location'].fillna('')
    cleaned_df['location'] = cleaned_df['location'].str.strip()
    
    # Clean description
    cleaned_df['description'] = cleaned_df['description'].fillna('')
    cleaned_df['description'] = cleaned_df['description'].str.strip()
    
    # Add source identifier
    cleaned_df['source'] = 'google_calendar'
    
    # Remove rows with invalid start datetime
    cleaned_df = cleaned_df.dropna(subset=['start_datetime'])
    
    return cleaned_df[['title', 'start_datetime', 'end_datetime', 'location', 'description', 'source']]


def clean_web_scraping_df(df):
    """
    Clean and standardize web scraping DataFrame
    
    Parameters:
    df (pd.DataFrame): Raw web scraping DataFrame with columns: title, link, date_time, venue, address
    
    Returns:
    pd.DataFrame: Cleaned DataFrame with standardized columns
    """
    if df.empty:
        return pd.DataFrame(columns=['title', 'start_datetime', 'end_datetime', 'location', 'description', 'source'])
    
    # Create a copy to avoid modifying original
    cleaned_df = df.copy()
    
    # Clean title
    cleaned_df['title'] = cleaned_df['title'].fillna('Untitled Event')
    cleaned_df['title'] = cleaned_df['title'].str.strip()
    
    # Parse date_time column
    def parse_datetime_string(date_str):
        """Parse various datetime formats from web scraping"""
        if pd.isna(date_str) or not date_str:
            return None, None
            
        try:
            # Handle format like "Saturday, October 4 · 10:15 - 11:15am EDT"
            if '·' in date_str and '-' in date_str and any(day in date_str for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']):
                # Split by the middle dot
                date_part, time_part = date_str.split('·', 1)
                date_part = date_part.strip()
                time_part = time_part.strip()
                
                # Extract and remove timezone if present
                timezone = None
                for tz in ['EDT', 'EST', 'CDT', 'CST', 'MDT', 'MST', 'PDT', 'PST', 'UTC']:
                    if tz in time_part:
                        timezone = tz
                        time_part = time_part.replace(tz, '').strip()
                        break
                
                # Parse time range (e.g., "10:15 - 11:15am")
                if '-' in time_part:
                    start_time_str, end_time_str = time_part.split('-', 1)
                    start_time_str = start_time_str.strip()
                    end_time_str = end_time_str.strip()
                    
                    # Handle am/pm - if end time has am/pm but start doesn't, apply appropriate am/pm to start
                    if ('am' in end_time_str or 'pm' in end_time_str) and not ('am' in start_time_str or 'pm' in start_time_str):
                        # Extract am/pm from end time
                        am_pm = 'am' if 'am' in end_time_str else 'pm'
                        start_time_str += am_pm
                    
                    # Get current year for parsing
                    current_year = datetime.now().year
                    
                    try:
                        # Remove day of week and parse date
                        date_without_day = re.sub(r'^[A-Za-z]+,\s*', '', date_part)
                        # Add current year if not present
                        if str(current_year) not in date_without_day:
                            date_without_day += f', {current_year}'
                        
                        base_date = pd.to_datetime(date_without_day, errors='coerce')
                        if pd.isna(base_date):
                            return None, None
                        
                        # Combine date with times
                        start_datetime_str = f"{base_date.strftime('%Y-%m-%d')} {start_time_str}"
                        end_datetime_str = f"{base_date.strftime('%Y-%m-%d')} {end_time_str}"
                        
                        start_dt = pd.to_datetime(start_datetime_str, errors='coerce', utc=True)
                        end_dt = pd.to_datetime(end_datetime_str, errors='coerce', utc=True)
                        
                        return start_dt, end_dt
                    except Exception as e:
                        return None, None
            
            # Handle format like "2024-01-15T18:00:00Z → 2024-01-15T20:00:00Z"
            elif '→' in date_str:
                start_str, end_str = date_str.split('→')
                start_dt = pd.to_datetime(start_str.strip(), errors='coerce', utc=True)
                end_dt = pd.to_datetime(end_str.strip(), errors='coerce', utc=True)
                return start_dt, end_dt
            else:
                # Single datetime
                start_dt = pd.to_datetime(date_str.strip(), errors='coerce', utc=True)
                return start_dt, None
        except Exception as e:
            return None, None
    
    # Apply datetime parsing
    datetime_parsed = cleaned_df['date_time'].apply(parse_datetime_string)
    cleaned_df['start_datetime'] = [x[0] for x in datetime_parsed]
    cleaned_df['end_datetime'] = [x[1] for x in datetime_parsed]
    
    # Process location information
    def format_location(venue, address):
        """Combine venue and address information"""
        location_parts = []
        
        if pd.notna(venue) and venue.strip():
            location_parts.append(venue.strip())
        
        if pd.notna(address) and address:
            if isinstance(address, dict):
                # Extract address components from dictionary
                addr_parts = []
                for key in ['streetAddress', 'addressLocality', 'addressRegion', 'postalCode']:
                    if key in address and address[key]:
                        addr_parts.append(str(address[key]))
                if addr_parts:
                    location_parts.append(', '.join(addr_parts))
            elif isinstance(address, str):
                location_parts.append(address.strip())
        
        return ' - '.join(location_parts) if location_parts else ''
    
    cleaned_df['location'] = cleaned_df.apply(lambda row: format_location(row.get('venue'), row.get('address')), axis=1)
    
    # Create description from link
    cleaned_df['description'] = cleaned_df['link'].fillna('')
    
    # Add source identifier
    cleaned_df['source'] = 'web_scraping'
    
    # Remove rows with invalid start datetime
    cleaned_df = cleaned_df.dropna(subset=['start_datetime'])
    
    return cleaned_df[['title', 'start_datetime', 'end_datetime', 'location', 'description', 'source']]


def combine_and_clean_dataframes(google_df=None, web_scraping_df=None):
    """
    Combine and clean both Google Calendar and web scraping DataFrames
    
    Parameters:
    google_df (pd.DataFrame, optional): Raw Google Calendar DataFrame
    web_scraping_df (pd.DataFrame, optional): Raw web scraping DataFrame
    
    Returns:
    pd.DataFrame: Combined and cleaned DataFrame
    """
    cleaned_dfs = []
    
    # Clean Google Calendar data if provided
    if google_df is not None and not google_df.empty:
        cleaned_google = clean_google_calendar_df(google_df)
        if not cleaned_google.empty:
            cleaned_dfs.append(cleaned_google)
    
    # Clean web scraping data if provided
    if web_scraping_df is not None and not web_scraping_df.empty:
        cleaned_web = clean_web_scraping_df(web_scraping_df)
        if not cleaned_web.empty:
            cleaned_dfs.append(cleaned_web)
    
    # Combine all cleaned dataframes
    if not cleaned_dfs:
        return pd.DataFrame(columns=['title', 'start_datetime', 'end_datetime', 'location', 'description', 'source'])
    
    combined_df = pd.concat(cleaned_dfs, ignore_index=True)
    
    # Sort by start datetime
    combined_df = combined_df.sort_values('start_datetime').reset_index(drop=True)
    
    # Remove exact duplicates
    combined_df = combined_df.drop_duplicates().reset_index(drop=True)
    
    return combined_df


In [20]:
# Create comprehensive sample data and test all functions
print("TESTING DATA CLEANING FUNCTIONS")

# Sample Google Calendar data with various edge cases
sample_google_data = {
    'Summary': [
        'Morning Yoga Class',
        'Team Meeting - Weekly Sync', 
        'Gym Workout Session',
        None,  # Test missing title
        'Pilates & Stretching',
        'Basketball Practice'
    ],
    'Start': [
        '2024-01-15T08:00:00Z',           # UTC format
        '2024-01-15T10:00:00-05:00',     # EST timezone
        '2024-01-16T18:00:00Z',          # Evening event
        '2024-01-17T12:00:00Z',          # Missing title test
        '2024-01-18T07:30:00-08:00',     # PST timezone
        '2024-01-19T16:00:00Z'           # Weekend event
    ],
    'End': [
        '2024-01-15T09:00:00Z',
        '2024-01-15T11:00:00-05:00',
        '2024-01-16T19:30:00Z',
        '2024-01-17T13:00:00Z',
        '2024-01-18T08:30:00-08:00',
        '2024-01-19T18:00:00Z'
    ],
    'Location': [
        'Yoga Studio Downtown',
        '',                               # Empty location
        'Gold\'s Gym - Main Street',
        'Community Center',
        'Pilates Plus Studio',
        'Outdoor Basketball Court'
    ],
    'Description': [
        'Beginner-friendly hatha yoga class',
        'Weekly team sync and project updates',
        '',                               # Empty description
        'Pickup basketball game - all welcome',
        'Advanced pilates with equipment',
        'Team practice for upcoming tournament'
    ]
}

sample_google_df = pd.DataFrame(sample_google_data)

# Sample web scraping data with various datetime formats
sample_web_data = {
    'title': [
        'High-Intensity CrossFit',
        'Dance Fitness Fusion', 
        'Outdoor Boot Camp',
        'Spin Class - Beginner',
        'Zumba Party Night',
        'Martial Arts Training'
    ],
    'link': [
        'https://eventbrite.com/e/crossfit-1234',
        'https://eventbrite.com/e/dance-5678',
        'https://eventbrite.com/e/bootcamp-9012',
        'https://eventbrite.com/e/spin-3456',
        'https://eventbrite.com/e/zumba-7890',
        'https://eventbrite.com/e/martial-arts-2468'
    ],
    'date_time': [
        # Test different datetime formats
        '2024-01-20T19:00:00Z → 2024-01-20T20:30:00Z',     # ISO with arrow
        'Saturday, January 21 · 10:00 - 11:30am EST',      # New format with timezone
        'Sunday, January 22 · 8:00 - 9:00am PST',          # Weekend morning
        'Monday, January 23 · 6:30 - 7:30pm EDT',          # Evening class
        'Friday, January 24 · 7:00 - 8:30pm CST',          # Friday night
        '2024-01-25T17:00:00Z'                              # Single datetime
    ],
    'venue': [
        'CrossFit Downtown',
        'Dance Academy Plus',
        'Riverside Park',
        'Spin Studio Central',
        'Community Dance Hall',
        'Martial Arts Dojo'
    ],
    'address': [
        # Test different address formats
        {'streetAddress': '123 Fitness Blvd', 'addressLocality': 'Pittsburgh', 'addressRegion': 'PA', 'postalCode': '15213'},
        {'streetAddress': '456 Dance Ave', 'addressLocality': 'Pittsburgh', 'addressRegion': 'PA'},  # Missing postal code
        'Riverside Park, 789 Park Drive, Pittsburgh PA',     # String format
        {'streetAddress': '321 Spin St', 'addressLocality': 'Pittsburgh', 'addressRegion': 'PA', 'postalCode': '15232'},
        'Downtown Community Center, Pittsburgh PA 15219',    # Another string format
        {'streetAddress': '654 Martial Way', 'addressLocality': 'Pittsburgh', 'addressRegion': 'PA', 'postalCode': '15206'}
    ]
}

sample_web_df = pd.DataFrame(sample_web_data)

print("SAMPLE DATA CREATED")
print("-" * 30)
print(f"Google Calendar sample: {sample_google_df.shape[0]} events")
print(f"Web scraping sample: {sample_web_df.shape[0]} events")
print()

# Display sample data
print("GOOGLE CALENDAR SAMPLE DATA:")
print(sample_google_df.to_string(index=False))
print()

print("WEB SCRAPING SAMPLE DATA:")
print(sample_web_df.to_string(index=False))

TESTING DATA CLEANING FUNCTIONS
SAMPLE DATA CREATED
------------------------------
Google Calendar sample: 6 events
Web scraping sample: 6 events

GOOGLE CALENDAR SAMPLE DATA:
                   Summary                     Start                       End                 Location                           Description
        Morning Yoga Class      2024-01-15T08:00:00Z      2024-01-15T09:00:00Z     Yoga Studio Downtown    Beginner-friendly hatha yoga class
Team Meeting - Weekly Sync 2024-01-15T10:00:00-05:00 2024-01-15T11:00:00-05:00                           Weekly team sync and project updates
       Gym Workout Session      2024-01-16T18:00:00Z      2024-01-16T19:30:00Z Gold's Gym - Main Street                                      
                      None      2024-01-17T12:00:00Z      2024-01-17T13:00:00Z         Community Center  Pickup basketball game - all welcome
      Pilates & Stretching 2024-01-18T07:30:00-08:00 2024-01-18T08:30:00-08:00      Pilates Plus Studio       Adva

In [22]:
print("\n" + "="*60)
print("TESTING INDIVIDUAL CLEANING FUNCTIONS")
print("="*60)

# Test 1: Clean Google Calendar Data
print("\nTESTING: clean_google_calendar_df()")
print("-" * 40)
cleaned_google = clean_google_calendar_df(sample_google_df)
print(f"Input shape: {sample_google_df.shape}")
print(f"Output shape: {cleaned_google.shape}")
print(f"Columns: {list(cleaned_google.columns)}")
print("\nCleaned Google Calendar Data:")
print(cleaned_google.to_string(index=False))

# Check data quality
print(f"\nGoogle Calendar Data Quality:")
print(f"   • Events with valid start times: {len(cleaned_google)}")
print(f"   • Events with end times: {cleaned_google['end_datetime'].notna().sum()}")
print(f"   • Events with locations: {(cleaned_google['location'] != '').sum()}")
print(f"   • Events with descriptions: {(cleaned_google['description'] != '').sum()}")

print("\n" + "="*60)

# Test 2: Clean Web Scraping Data
print("\nTESTING: clean_web_scraping_df()")
print("-" * 40)
cleaned_web = clean_web_scraping_df(sample_web_df)
print(f"Input shape: {sample_web_df.shape}")
print(f"Output shape: {cleaned_web.shape}")
print(f"Columns: {list(cleaned_web.columns)}")
print("\nCleaned Web Scraping Data:")
print(cleaned_web.to_string(index=False))

# Check data quality and datetime parsing
print(f"\nWeb Scraping Data Quality:")
print(f"   • Events with valid start times: {len(cleaned_web)}")
print(f"   • Events with end times: {cleaned_web['end_datetime'].notna().sum()}")
print(f"   • Events with locations: {(cleaned_web['location'] != '').sum()}")

# Show datetime parsing results
print(f"\nDateTime Parsing Results:")
for i, row in cleaned_web.iterrows():
    duration = row['end_datetime'] - row['start_datetime'] if pd.notna(row['end_datetime']) else None
    print(f"   • {row['title'][:25]:.<25} {str(duration).split()[2] if duration else 'No end time'}")

print("\n" + "="*60)


TESTING INDIVIDUAL CLEANING FUNCTIONS

TESTING: clean_google_calendar_df()
----------------------------------------
Input shape: (6, 5)
Output shape: (6, 6)
Columns: ['title', 'start_datetime', 'end_datetime', 'location', 'description', 'source']

Cleaned Google Calendar Data:
                     title            start_datetime              end_datetime                 location                           description          source
        Morning Yoga Class 2024-01-15 08:00:00+00:00 2024-01-15 09:00:00+00:00     Yoga Studio Downtown    Beginner-friendly hatha yoga class google_calendar
Team Meeting - Weekly Sync 2024-01-15 15:00:00+00:00 2024-01-15 16:00:00+00:00                           Weekly team sync and project updates google_calendar
       Gym Workout Session 2024-01-16 18:00:00+00:00 2024-01-16 19:30:00+00:00 Gold's Gym - Main Street                                       google_calendar
            Untitled Event 2024-01-17 12:00:00+00:00 2024-01-17 13:00:00+00:00         Co

In [23]:
print("\nTESTING: combine_and_clean_dataframes()")
print("-" * 40)

# Test the combined function
combined_data = combine_and_clean_dataframes(sample_google_df, sample_web_df)
print(f"Combined shape: {combined_data.shape}")
print(f"Total events: {len(combined_data)}")

# Show source distribution
source_counts = combined_data['source'].value_counts()
print(f"Source distribution: {dict(source_counts)}")

print("\nCOMBINED & CLEANED DATA (sorted by datetime):")
print(combined_data.to_string(index=False))

# Final data quality summary
print(f"\nFINAL DATA QUALITY SUMMARY:")
print(f"   • Total events: {len(combined_data)}")
print(f"   • Date range: {combined_data['start_datetime'].min().strftime('%Y-%m-%d')} to {combined_data['start_datetime'].max().strftime('%Y-%m-%d')}")
print(f"   • Events with end times: {combined_data['end_datetime'].notna().sum()}")
print(f"   • Events with locations: {(combined_data['location'] != '').sum()}")
print(f"   • Events with descriptions: {(combined_data['description'] != '').sum()}")
print(f"   • Google Calendar events: {len(combined_data[combined_data['source'] == 'google_calendar'])}")
print(f"   • Web scraping events: {len(combined_data[combined_data['source'] == 'web_scraping'])}")

# Test edge cases
print(f"\nEDGE CASE TESTING:")
print("-" * 20)

# Test with empty dataframes
empty_result = combine_and_clean_dataframes(pd.DataFrame(), pd.DataFrame())
print(f"Empty DataFrames test: {empty_result.shape} -> {list(empty_result.columns)}")

# Test with None inputs
none_result = combine_and_clean_dataframes(None, None)
print(f"None inputs test: {none_result.shape} -> {list(none_result.columns)}")

# Test with only one source
google_only = combine_and_clean_dataframes(sample_google_df, None)
print(f"Google only test: {google_only.shape}")

web_only = combine_and_clean_dataframes(None, sample_web_df)
print(f"Web scraping only test: {web_only.shape}")



TESTING: combine_and_clean_dataframes()
----------------------------------------
Combined shape: (12, 6)
Total events: 12
Source distribution: {'google_calendar': np.int64(6), 'web_scraping': np.int64(6)}

COMBINED & CLEANED DATA (sorted by datetime):
                     title            start_datetime              end_datetime                                                              location                                description          source
        Morning Yoga Class 2024-01-15 08:00:00+00:00 2024-01-15 09:00:00+00:00                                                  Yoga Studio Downtown         Beginner-friendly hatha yoga class google_calendar
Team Meeting - Weekly Sync 2024-01-15 15:00:00+00:00 2024-01-15 16:00:00+00:00                                                                             Weekly team sync and project updates google_calendar
       Gym Workout Session 2024-01-16 18:00:00+00:00 2024-01-16 19:30:00+00:00                                             